### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#Observations are at the bottom of the worksheet

# Dependencies and Setup
import pandas as pd
import numpy as np
import os
# File to Load (Remember to Change These)
file_to_load = os.path.join("purchase_data.csv")
# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
#View first rows in the file
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
#Get the unique count of players and put into a summary DF to display
total_players = len(purchase_data_df["SN"].unique())
summary_tp_df = pd.DataFrame({"Total Players": [total_players]})
summary_tp_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#See what is in items column
purchase_data_df["Item ID"].value_counts()
#Calculations to obtain unique count, average price, number of pruchases and total revenue 
unique_items = len(purchase_data_df["Item ID"].unique())
avg_price = purchase_data_df["Price"].mean()
number_purchases = len(purchase_data_df["Purchase ID"].unique())
total_revenue = purchase_data_df["Price"].sum()
#Format Average Price and Total Revenue to Currency
avg_price = '${:.3}'.format(avg_price)
total_revenue = '${:,}'.format(total_revenue)
#Create summary table to display
summary_item_df = pd.DataFrame({"Number of Unique Items": [unique_items], "Average Price": [avg_price], 
                                "Number of Purchases": [number_purchases], "Total Revenue": [total_revenue]})
summary_item_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#Remove Dupcliate players and get unique count
no_duplicates = purchase_data_df.drop_duplicates("SN")
ttl_players = len(no_duplicates["SN"].unique())
#Set group to Gender and add column count
group_gender_df= no_duplicates.groupby(["Gender"]).size().reset_index(name='counts')
#Set Gender to index 
group_gender_df = group_gender_df.set_index("Gender")
#Use loc to get make each gender count a variable
male_ttl = group_gender_df.loc["Male", "counts"]
female_ttl = group_gender_df.loc["Female", "counts"]
other_ttl = group_gender_df.loc["Other / Non-Disclosed", "counts"]
#Get the % of gender
male_percentage =(male_ttl / ttl_players)*100
female_percentage =(female_ttl / ttl_players)*100
other_percentage  =(other_ttl / ttl_players)*100
#Format Average Price and Total Revenue to Currency
male_percentage = '{:.4}'.format(male_percentage)
female_percentage = '{:.4}'.format(female_percentage)
other_percentage = '{:.4}'.format(other_percentage)
#Summary table to display
summary_gender_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"],
                                  "Total Count": [male_ttl, female_ttl, other_ttl], 
                                  "Percentage of Players": [male_percentage, female_percentage, other_percentage]})
#Remove the index from Summary Table
blankIndex=[''] * len(summary_gender_df)
summary_gender_df.index=blankIndex
summary_gender_df

,Gender,Total Count,Percentage of Players
,Male,484,84.03
,Female,81,14.06
,Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Get Gender groupby 
gender_purchase = purchase_data_df.groupby(["Gender"])
gender_compare = gender_purchase.count()
# Calculate purchase count, avg price by gender
gender_compare["Purchase Count"] = gender_purchase["Purchase ID"].count()
gender_compare["Average Purchase Price"] = gender_purchase["Price"].mean()
gender_compare["Total Purchase Value"] = gender_purchase["Price"].sum()
#Calculate total purchase per person
gender_compare["unique_ppl"] = [female_ttl, male_ttl,  other_ttl]
gender_compare["Avg Total Purchase per Person"] = gender_compare["Total Purchase Value"] / gender_compare["unique_ppl"]
#Format Average Price,Total Revenue and Avg Per Person to Currency
gender_compare["Average Purchase Price"] = gender_compare["Average Purchase Price"].map("${:.2f}".format)
gender_compare["Total Purchase Value"] = gender_compare["Total Purchase Value"].map("${:.2f}".format)
gender_compare["Avg Total Purchase per Person"] = gender_compare["Avg Total Purchase per Person"].map("${:.2f}".format)
#Reduce dataframe to only need columns
gender_purchase_reduce = gender_compare[[ "Purchase Count","Average Purchase Price", 
                                         "Total Purchase Value", "Avg Total Purchase per Person" ]]
gender_purchase_reduce

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Create the bins in which Data will be held
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
# Create the names for the four bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#Creates a new cloumn with Age Groups bins and label=group_names
no_duplicates["Age Groups"] = pd.cut(no_duplicates["Age"], bins, labels=group_names)
#Group data by Age Groups
age_group = no_duplicates.groupby(["Age Groups"])
age_compare = age_group.count()
#Create a new comlumn for Total Count
age_compare["Total Count"] = age_group["Age"].count()
age_unique = len(no_duplicates["Age"])
#Create column for Percentage of Player and format
age_compare["Percentage of Players"] = ((age_compare["Total Count"] / age_unique)*100).map("{:.3}".format)
#Create summary df and display
summary_age_df = age_compare[[ "Total Count", "Percentage of Players"]]
summary_age_df



C:\Users\Marc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Total Count,Percentage of Players
Age Groups,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.6
20-24,258,44.8
25-29,77,13.4
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Create the bins in which Data will be held
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
# Create the names for the four bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#Creates a new cloumn with Age Groups bins and label=group_names
purchase_data_df["Age Groups"] = pd.cut(purchase_data_df["Age"], bins, labels=group_names)
#Create gorupby by Age Groups
age_purchase = purchase_data_df.groupby(["Age Groups"])
age_data = age_purchase.count()
# Calculate purchase count, avg price by gender
age_data["Purchase Count"] = age_purchase["Purchase ID"].count()
age_data["Average Purchase Price"] = age_purchase["Price"].mean()
age_data["Total Purchase Value"] = age_purchase["Price"].sum()
#Calculate total purchase per person
age_data["Total Count"] = age_group["Age"].count()
age_data["Avg Total Purchase per Person"] = age_data["Total Purchase Value"] / age_data["Total Count"]
#Format Average Price,Total Revenue and Avg Per Person to Currency
age_data["Average Purchase Price"] = age_data["Average Purchase Price"].map("${:.2f}".format)
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map("${:.2f}".format)
age_data["Avg Total Purchase per Person"] = age_data["Avg Total Purchase per Person"].map("${:.2f}".format)
#Reduce dataframe to only need columns
age_purchase_reduce = age_data[[ "Purchase Count","Average Purchase Price", 
                                         "Total Purchase Value", "Avg Total Purchase per Person" ]]
age_purchase_reduce



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Group data by Player Name
player_group = purchase_data_df.groupby(["SN"])
player_data = player_group.count()
# Calculate purchase count, avg price by player
player_data["Purchase Count"] = player_group["Purchase ID"].count()
player_data["Average Purchase Price"] = player_group["Price"].mean()
player_data["Total Purchase Value"] = player_group["Price"].sum()
#Sort data by Total Purchase Value
player_data = player_data.sort_values("Total Purchase Value", ascending=False)
#Format columns to currency
player_data["Average Purchase Price"] = player_data["Average Purchase Price"].map("${:.2f}".format)
player_data["Total Purchase Value"] = player_data["Total Purchase Value"].map("${:.2f}".format)
#Reduce data to needed columns
sn_purchase_reduce = player_data[[ "Average Purchase Price", "Total Purchase Value" ]]

sn_purchase_reduce.head()

,Average Purchase Price,Total Purchase Value
SN,,
Lisosia93,$3.79,$18.96
Idastidru52,$3.86,$15.45
Chamjask73,$4.61,$13.83
Iral74,$3.40,$13.62
Iskadarya95,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#Group data by both Item ID and Item Name
item_group = purchase_data_df.groupby(["Item ID", "Item Name"])
item_data = item_group.count()
# Calculate purchase count, avg price by item
item_data["Purchase Count"] = item_group["Purchase ID"].count()
item_data["Average Purchase Price"] = item_group["Price"].mean()
item_data["Total Purchase Value"] = item_group["Price"].sum()
#Sort Items by Purchase Count
item_data = item_data.sort_values("Purchase Count", ascending=False) 
#Format to currency
item_data["Average Purchase Price"] = item_data["Average Purchase Price"].map("${:.2f}".format)
item_data["Total Purchase Value"] = item_data["Total Purchase Value"].map("${:.2f}".format)
#reduce DF to only need columns
item_purchase_reduce = item_data[[ "Purchase Count", "Average Purchase Price", "Total Purchase Value" ]]

item_purchase_reduce.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [23]:
#Group data by both Item ID and Item Name
item_group = purchase_data_df.groupby(["Item ID", "Item Name"])
item_data = item_group.count()
# Calculate purchase count, avg price by item
item_data["Purchase Count"] = item_group["Purchase ID"].count()
item_data["Average Purchase Price"] = item_group["Price"].mean()
item_data["Total Purchase Value"] = item_group["Price"].sum()
#Sort data by total purchase
item_data = item_data.sort_values("Total Purchase Value", ascending=False) 
#format to currency
item_data["Average Purchase Price"] = item_data["Average Purchase Price"].map("${:.2f}".format)
item_data["Total Purchase Value"] = item_data["Total Purchase Value"].map("${:.2f}".format)
#reduce df to columns needed
item_purchase_reduce = item_data[[ "Purchase Count", "Average Purchase Price", "Total Purchase Value" ]]

item_purchase_reduce.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [14]:
#Additional Analysis - Find new info by grouping Age and Gender

age_gender_group2 = purchase_data_df.groupby(["Age Groups", "Gender"])
age_gender_data2 = age_gender_group2.count()
#Calcuated needed values
age_gender_data2["Purchase Count"] = age_gender_group2["Purchase ID"].count()
age_gender_data2["Average Purchase Price"] = age_gender_group2["Price"].mean()
age_gender_data2["Total Purchase Value"] = age_gender_group2["Price"].sum()

# Replace all NaN values with 0 
age_gender_data2 = age_gender_data2.fillna(0)
#format
age_gender_data2["Purchase Count"] = age_gender_data2["Purchase Count"].map("{:.0f}".format)
age_gender_data2["Average Purchase Price"] = age_gender_data2["Average Purchase Price"].map("${:.2f}".format)
age_gender_data2["Total Purchase Value"] = age_gender_data2["Total Purchase Value"].map("${:.2f}".format)
#create df
age_gender_purchase_reduce = age_gender_data2[[ "Purchase Count", "Average Purchase Price", "Total Purchase Value" ]]

age_gender_purchase_reduce

Purchase Count Average Purchase Price  \
Age Groups Gender                                                        
<10        Female                             5                  $3.64   
           Male                              18                  $3.27   
           Other / Non-Disclosed              0                  $0.00   
10-14      Female                             3                  $3.61   
           Male                              25                  $2.88   
           Other / Non-Disclosed              0                  $0.00   
15-19      Female                            24                  $3.46   
           Male                             108                  $2.94   
           Other / Non-Disclosed              4                  $2.97   
20-24      Female                            56                  $3.13   
           Male                             304                  $3.03   
           Other / Non-Disclosed              5                  $3.72   
25-29      Female                            14                  $2.83   
           Male                              85                  $2.92   
           Other / Non-Disclosed              2                  $2.64   
30-34      Female                             7                  $2.82   
           Male                              66                  $2.94   
           Other / Non-Disclosed              0                  $0.00   
35-39      Female                             3                  $3.55   
           Male                              34                  $3.61   
           Other / Non-Disclosed              4                  $3.61   
40+        Female                             1                  $4.40   
           Male                              12                  $2.82   
           Other / Non-Disclosed              0                  $0.00   

                                 Total Purchase Value  
Age Groups Gender                                      
<10        Female                              $18.19  
           Male                                $58.94  
           Other / Non-Disclosed                $0.00  
10-14      Female                              $10.82  
           Male                                $71.96  
           Other / Non-Disclosed                $0.00  
15-19      Female                              $82.96  
           Male                               $318.04  
           Other / Non-Disclosed               $11.89  
20-24      Female                             $175.51  
           Male                               $919.96  
           Other / Non-Disclosed               $18.59  
25-29      Female                              $39.68  
           Male                               $248.04  
           Other / Non-Disclosed                $5.28  
30-34      Female                              $19.72  
           Male                               $194.28  
           Other / Non-Disclosed                $0.00  
35-39      Female                              $10.66  
           Male                               $122.58  
           Other / Non-Disclosed               $14.43  
40+        Female                               $4.40  
           Male                                $33.84  
           Other / Non-Disclosed                $0.00

In [18]:
#Additional Analysis - Grouping Players and Gender
player_group2 = purchase_data_df.groupby(["SN", "Gender"])
player_data2 = player_group2.count()

# Calculate purchase count, avg price by gender
player_data2["Purchase Count"] = player_group2["Purchase ID"].count()
player_data2["Average Purchase Price"] = player_group2["Price"].mean()
player_data2["Total Purchase Value"] = player_group2["Price"].sum()
player_data2 = player_data2.sort_values("Total Purchase Value", ascending=False)

player_data2["Average Purchase Price"] = player_data2["Average Purchase Price"].map("${:.2f}".format)
player_data2["Total Purchase Value"] = player_data2["Total Purchase Value"].map("${:.2f}".format)

sn_purchase_reduce2 = player_data2[[ "Purchase Count", "Average Purchase Price", "Total Purchase Value" ]]

sn_purchase_reduce2.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
SN,Gender,,,
Lisosia93,Male,5,$3.79,$18.96
Idastidru52,Male,4,$3.86,$15.45
Chamjask73,Female,3,$4.61,$13.83
Iral74,Male,4,$3.40,$13.62
Iskadarya95,Male,3,$4.37,$13.10


In [21]:
#Additional Analysis - Grouping Player and Age
player_group3 = purchase_data_df.groupby(["SN", "Age"])
player_data3 = player_group3.count()

# Calculate purchase count, avg price by gender
player_data3["Purchase Count"] = player_group3["Purchase ID"].count()
player_data3["Average Purchase Price"] = player_group3["Price"].mean()
player_data3["Total Purchase Value"] = player_group3["Price"].sum()
player_data3 = player_data3.sort_values("Total Purchase Value", ascending=False)

player_data3["Average Purchase Price"] = player_data3["Average Purchase Price"].map("${:.2f}".format)
player_data3["Total Purchase Value"] = player_data3["Total Purchase Value"].map("${:.2f}".format)

sn_purchase_reduce3 = player_data3[[ "Purchase Count", "Average Purchase Price", "Total Purchase Value" ]]

sn_purchase_reduce3.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
SN,Age,,,
Lisosia93,25,5,$3.79,$18.96
Idastidru52,24,4,$3.86,$15.45
Chamjask73,22,3,$4.61,$13.83
Iral74,21,4,$3.40,$13.62
Iskadarya95,20,3,$4.37,$13.10


Oberservations
1. As stated in the beginning males comprise 84% of the users, they also account for 82% of the
    purchases by value and 83.5%, which is consistent with the overall players
2. While the number females comprise only 14% of players and around that in sales, they actually
    avg higher per sale, $4.47, than males, $4.07.
3. in looking at the top 5 players by spending, it was interesting that a female in the top 5, 
    and she had the highest per sale avg. Also when looking attop spenders and age, it is not 
    suprising that 4 out of the 5 where, in 20-24 age group since it was the highest spending age
    group. The one outlier was actual the highest spender overall, and was in the 25-29 age group.